# Handling High cardinality solution
- Too many unique values of categorical variable-
-  범주형변수의 고유값이너무 많은 롱테일 형태의 분포

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings(action='ignore') 

import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
# 데이터 생성
data = {'온도' : ['Hot', 'Cold', 'Very Hot', 'Warm', 'Hot', 'Warm', 'Warm', 'Hot', 'Hot', 'Cold'],
        '색상' : ['Red', 'Yellow', 'Blue', 'Blue', 'Red', 'Yellow', 'Red', 'Yellow', 'Yellow', 'Yellow'],
        'target' : [1,1,1,0,1,0,1,0,1,1]}  
df = pd.DataFrame(data)
df

,온도,색상,target
0,Hot,Red,1
1,Cold,Yellow,1
2,Very Hot,Blue,1
3,Warm,Blue,0
4,Hot,Red,1
5,Warm,Yellow,0
6,Warm,Red,1
7,Hot,Yellow,0
8,Hot,Yellow,1
9,Cold,Yellow,1


# 1. One Hot Encoding

In [20]:
pd.get_dummies(df, columns=['온도', '색상'])

,target,온도_Cold,온도_Hot,온도_Very Hot,온도_Warm,색상_Blue,색상_Red,색상_Yellow
0,1,0,1,0,0,0,1,0
1,1,1,0,0,0,0,0,1
2,1,0,0,1,0,1,0,0
3,0,0,0,0,1,1,0,0
4,1,0,1,0,0,0,1,0
5,0,0,0,0,1,0,0,1
6,1,0,0,0,1,0,1,0
7,0,0,1,0,0,0,0,1
8,1,0,1,0,0,0,0,1
9,1,1,0,0,0,0,0,1


In [12]:
df.온도
df.온도.shape
df.온도.values
df.온도.values.reshape(-1,1)

0         Hot
1        Cold
2    Very Hot
3        Warm
4         Hot
5        Warm
6        Warm
7         Hot
8         Hot
9        Cold
Name: 온도, dtype: object

(10,)

array(['Hot', 'Cold', 'Very Hot', 'Warm', 'Hot', 'Warm', 'Warm', 'Hot',
       'Hot', 'Cold'], dtype=object)

array([['Hot'],
       ['Cold'],
       ['Very Hot'],
       ['Warm'],
       ['Hot'],
       ['Warm'],
       ['Warm'],
       ['Hot'],
       ['Hot'],
       ['Cold']], dtype=object)

In [13]:
df.온도.values.reshape(-1,1).shape

(10, 1)

In [15]:
from sklearn.preprocessing import OneHotEncoder
ohc = OneHotEncoder()
ohe = ohc.fit_transform(df.온도.values.reshape(-1,1)).toarray()
ohe

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [17]:
ohc.categories_
ohc.categories_[0][0]

[array(['Cold', 'Hot', 'Very Hot', 'Warm'], dtype=object)]

'Cold'

In [18]:
tf = pd.DataFrame(ohe, columns=['온도_'+ str(ohc.categories_[0][i])
                                for i in range(len(ohc.categories_[0]))]
                 )
tf

,온도_Cold,온도_Hot,온도_Very Hot,온도_Warm
0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,1.0
7,0.0,1.0,0.0,0.0
8,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,0.0


In [7]:
pd.concat([df, tf], axis=1)

,온도,색상,target,온도_Cold,온도_Hot,온도_Very Hot,온도_Warm
0,Hot,Red,1,0.0,1.0,0.0,0.0
1,Cold,Yellow,1,1.0,0.0,0.0,0.0
2,Very Hot,Blue,1,0.0,0.0,1.0,0.0
3,Warm,Blue,0,0.0,0.0,0.0,1.0
4,Hot,Red,1,0.0,1.0,0.0,0.0
5,Warm,Yellow,0,0.0,0.0,0.0,1.0
6,Warm,Red,1,0.0,0.0,0.0,1.0
7,Hot,Yellow,0,0.0,1.0,0.0,0.0
8,Hot,Yellow,1,0.0,1.0,0.0,0.0
9,Cold,Yellow,1,1.0,0.0,0.0,0.0


# 2. Label Encoding
### (Cold < Hot < Vey Hot < Warm...... 0 < 1 <2 <3)

In [21]:
from sklearn.preprocessing import LabelEncoder
la = LabelEncoder()
df['온도_label'] = LabelEncoder().fit_transform(df.온도)
df

,온도,색상,target,온도_label
0,Hot,Red,1,1
1,Cold,Yellow,1,0
2,Very Hot,Blue,1,2
3,Warm,Blue,0,3
4,Hot,Red,1,1
5,Warm,Yellow,0,3
6,Warm,Red,1,3
7,Hot,Yellow,0,1
8,Hot,Yellow,1,1
9,Cold,Yellow,1,0


### facrorization

In [9]:
df.loc[:, '온도_label'] = pd.factorize(df['온도'])[0]
df

,온도,색상,target,온도_label
0,Hot,Red,1,0
1,Cold,Yellow,1,1
2,Very Hot,Blue,1,2
3,Warm,Blue,0,3
4,Hot,Red,1,0
5,Warm,Yellow,0,3
6,Warm,Red,1,3
7,Hot,Yellow,0,0
8,Hot,Yellow,1,0
9,Cold,Yellow,1,1


# 3. Ordinal Encoding
### (Cold < Warm < Hot < Vey Hot < ...... 0 < 1 <2 <3)

In [10]:
temp_dict ={'Cold': 1, 'Warm': 2, 'Hot': 3, 'Very Hot': 4}
df['온도_ord'] = df.온도.map(temp_dict)
df

,온도,색상,target,온도_label,온도_ord
0,Hot,Red,1,0,3
1,Cold,Yellow,1,1,1
2,Very Hot,Blue,1,2,4
3,Warm,Blue,0,3,2
4,Hot,Red,1,0,3
5,Warm,Yellow,0,3,2
6,Warm,Red,1,3,2
7,Hot,Yellow,0,0,3
8,Hot,Yellow,1,0,3
9,Cold,Yellow,1,1,1


# 3. Frequency Encoding

In [11]:
fe = df.groupby('온도').size()/len(df)
df.loc[:, '온도_fe'] = df['온도'].map(fe)
df

,온도,색상,target,온도_label,온도_ord,온도_fe
0,Hot,Red,1,0,3,0.4
1,Cold,Yellow,1,1,1,0.2
2,Very Hot,Blue,1,2,4,0.1
3,Warm,Blue,0,3,2,0.3
4,Hot,Red,1,0,3,0.4
5,Warm,Yellow,0,3,2,0.3
6,Warm,Red,1,3,2,0.3
7,Hot,Yellow,0,0,3,0.4
8,Hot,Yellow,1,0,3,0.4
9,Cold,Yellow,1,1,1,0.2


# 4. Target Mean Encoding

## 4.1 mean-encoding

In [24]:
df

,온도,색상,target,온도_label
0,Hot,Red,1,1
1,Cold,Yellow,1,0
2,Very Hot,Blue,1,2
3,Warm,Blue,0,3
4,Hot,Red,1,1
5,Warm,Yellow,0,3
6,Warm,Red,1,3
7,Hot,Yellow,0,1
8,Hot,Yellow,1,1
9,Cold,Yellow,1,0


In [25]:
df.groupby(['온도'])['target'].mean()

온도
Cold        1.000000
Hot         0.750000
Very Hot    1.000000
Warm        0.333333
Name: target, dtype: float64

In [26]:
target_mean = df.groupby(['온도'])['target'].mean()
df['온도_tm'] = df['온도'].map(target_mean) # map 유용한 기능
df

,온도,색상,target,온도_label,온도_tm
0,Hot,Red,1,1,0.750000
1,Cold,Yellow,1,0,1.000000
2,Very Hot,Blue,1,2,1.000000
3,Warm,Blue,0,3,0.333333
4,Hot,Red,1,1,0.750000
5,Warm,Yellow,0,3,0.333333
6,Warm,Red,1,3,0.333333
7,Hot,Yellow,0,1,0.750000
8,Hot,Yellow,1,1,0.750000
9,Cold,Yellow,1,0,1.000000


## 4.2 smoothing

In [14]:
mean = df['target'].mean()
mean
agg = df.groupby('온도')['target'].agg(['count', 'mean'])
agg
agg['count'].sum()
agg['count']*agg['mean']/100

0.7

,count,mean
온도,,
Cold,2,1.000000
Hot,4,0.750000
Very Hot,1,1.000000
Warm,3,0.333333


10

온도
Cold        0.02
Hot         0.03
Very Hot    0.01
Warm        0.01
dtype: float64

In [15]:
mean = df['target'].mean()
mean
agg = df.groupby('온도')['target'].agg(['count', 'mean'])
counts = agg['count']
means = agg['mean']
weight = 100
smooth = (counts*means + weight*mean)/(counts+weight)
smooth

0.7

온도
Cold        0.705882
Hot         0.701923
Very Hot    0.702970
Warm        0.689320
dtype: float64

In [16]:
df.loc[:, '온도_smooth'] = df['온도'].map(smooth)
df

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth
0,Hot,Red,1,0,3,0.4,0.750000,0.701923
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320
4,Hot,Red,1,0,3,0.4,0.750000,0.701923
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320
6,Warm,Red,1,3,2,0.3,0.333333,0.689320
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882


## 4.3 multiple mean-encoding

In [27]:
m_tm = pd.DataFrame(df.groupby(['온도', '색상'])['target'].mean()).reset_index()
m_tm 

,온도,색상,target
0,Cold,Yellow,1.0
1,Hot,Red,1.0
2,Hot,Yellow,0.5
3,Very Hot,Blue,1.0
4,Warm,Blue,0.0
5,Warm,Red,1.0
6,Warm,Yellow,0.0


In [18]:
df = df.merge(m_tm, on=['온도', '색상'], how='left', suffixes=('', '_온도색상_tm'))
df

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth,target_온도색상_tm
0,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970,1.0
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320,0.0
4,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320,0.0
6,Warm,Red,1,3,2,0.3,0.333333,0.689320,1.0
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923,0.5
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923,0.5
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0


# 5. Supervised ratio

In [19]:
sr = df['온도'].value_counts()

In [20]:
df['온도_sr'] = df['온도'].map(sr)
df

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth,target_온도색상_tm,온도_sr
0,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970,1.0,1
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320,0.0,3
4,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320,0.0,3
6,Warm,Red,1,3,2,0.3,0.333333,0.689320,1.0,3
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923,0.5,4
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923,0.5,4
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2


# 6. weight of evidence
good/bad 비율, 즉 odd ratio의 자연로그 (odd ratio)를 사용, 결국 로짓회귀분석의 계수값

In [21]:
woe_df = pd.DataFrame(df.groupby('온도')['target'].mean()).rename(columns={'target': 'good'})
woe_df['bad'] = 1 - woe_df.good
woe_df['bad'] = np.where(woe_df['bad'] ==0, 0.000001, woe_df['bad']) # '0'으로 나누기 방지
woe_df['woe'] = np.log(woe_df.good/woe_df.bad)
woe_df

,good,bad,woe
온도,,,
Cold,1.000000,0.000001,13.815511
Hot,0.750000,0.250000,1.098612
Very Hot,1.000000,0.000001,13.815511
Warm,0.333333,0.666667,-0.693147


In [22]:
df['온도_woe'] = df['온도'].map(woe_df['woe'])
df

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth,target_온도색상_tm,온도_sr,온도_woe
0,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,1.098612
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,13.815511
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970,1.0,1,13.815511
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320,0.0,3,-0.693147
4,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,1.098612
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320,0.0,3,-0.693147
6,Warm,Red,1,3,2,0.3,0.333333,0.689320,1.0,3,-0.693147
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923,0.5,4,1.098612
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923,0.5,4,1.098612
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,13.815511


IV(information value)

In [23]:
woe_df['iv'] = (woe_df['good'] - woe_df['bad'])*woe_df['woe']
woe_df

,good,bad,woe,iv
온도,,,,
Cold,1.000000,0.000001,13.815511,13.815497
Hot,0.750000,0.250000,1.098612,0.549306
Very Hot,1.000000,0.000001,13.815511,13.815497
Warm,0.333333,0.666667,-0.693147,0.231049


In [24]:
woe_df['iv'].sum()

28.411348689428138

In [25]:
iv = woe_df['iv'].sum().round(2)
print(f'IV 값은 {iv}')  #  값이 클수록 해당 변수의 target에 대한 영향력이 크다

IV 값은 28.41


 IV 값을 범주형으로 만들어 더미변수로 변환 한후 로짓회귀분석 시행

In [26]:
tf = df[['온도', '색상', 'target', '온도_woe']]
tf.loc[:, '온도_woe'] = tf['온도_woe'].astype('category') 
tf

,온도,색상,target,온도_woe
0,Hot,Red,1,1.098612
1,Cold,Yellow,1,13.815511
2,Very Hot,Blue,1,13.815511
3,Warm,Blue,0,-0.693147
4,Hot,Red,1,1.098612
5,Warm,Yellow,0,-0.693147
6,Warm,Red,1,-0.693147
7,Hot,Yellow,0,1.098612
8,Hot,Yellow,1,1.098612
9,Cold,Yellow,1,13.815511


In [27]:
pd.get_dummies(df, prefix=['더미'], columns=['온도_woe'])

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth,target_온도색상_tm,온도_sr,더미_-0.6931471805599454,더미_1.0986122886681098,더미_13.815510557964274
0,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,0,1,0
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,0,0,1
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970,1.0,1,0,0,1
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320,0.0,3,1,0,0
4,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,0,1,0
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320,0.0,3,1,0,0
6,Warm,Red,1,3,2,0.3,0.333333,0.689320,1.0,3,1,0,0
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923,0.5,4,0,1,0
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923,0.5,4,0,1,0
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,0,0,1


# 7. probability ratio
WOE 방법과 동일하나 다만 good/bad 비율만 사용 (odd ratio)

In [28]:
pr_df = pd.DataFrame(df.groupby('온도')['target'].mean()).rename(columns={'target': 'good'})

pr_df['bad'] = 1 - pr_df.good
pr_df['bad'] = np.where(pr_df['bad'] ==0, 0.00001, pr_df['bad']) # '0'으로 나누기 방지
pr_df['PR'] = pr_df.good/pr_df.bad
pr_df

,good,bad,PR
온도,,,
Cold,1.000000,0.000010,100000.0
Hot,0.750000,0.250000,3.0
Very Hot,1.000000,0.000010,100000.0
Warm,0.333333,0.666667,0.5


In [29]:
df['온도_pr'] = df['온도'].map(pr_df['PR'])
df

,온도,색상,target,온도_label,온도_ord,온도_fe,온도_tm,온도_smooth,target_온도색상_tm,온도_sr,온도_woe,온도_pr
0,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,1.098612,3.0
1,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,13.815511,100000.0
2,Very Hot,Blue,1,2,4,0.1,1.000000,0.702970,1.0,1,13.815511,100000.0
3,Warm,Blue,0,3,2,0.3,0.333333,0.689320,0.0,3,-0.693147,0.5
4,Hot,Red,1,0,3,0.4,0.750000,0.701923,1.0,4,1.098612,3.0
5,Warm,Yellow,0,3,2,0.3,0.333333,0.689320,0.0,3,-0.693147,0.5
6,Warm,Red,1,3,2,0.3,0.333333,0.689320,1.0,3,-0.693147,0.5
7,Hot,Yellow,0,0,3,0.4,0.750000,0.701923,0.5,4,1.098612,3.0
8,Hot,Yellow,1,0,3,0.4,0.750000,0.701923,0.5,4,1.098612,3.0
9,Cold,Yellow,1,1,1,0.2,1.000000,0.705882,1.0,2,13.815511,100000.0


# End